In [1]:
import pandas as pd
import numpy as np

In [2]:
"""
Load data
"""
df_train = pd.read_csv('data/titanic/train.csv')
df_test = pd.read_csv('data/titanic/test.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
"""
Feature engineering
"""
from sklearn import preprocessing
from keras.utils import to_categorical

le = preprocessing.LabelEncoder()
mms = preprocessing.MinMaxScaler()

# Handle missing
df_train = df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1).dropna()
df_test = df_test.fillna(method='ffill')

# Embarked
embarked_train = to_categorical(le.fit_transform(df_train['Embarked']))
embarked_test = to_categorical(le.fit_transform(df_test['Embarked']))

# Fare
fare_train = df_train['Fare'].values.reshape(-1, 1)
fare_test = df_test['Fare'].values.reshape(-1, 1)

# Siblings
siblings_train = df_train['SibSp'].values.reshape(-1, 1)
siblings_test = df_test['SibSp'].values.reshape(-1, 1)

# Parents
parents_train = df_train['Parch'].values.reshape(-1, 1)
parents_test = df_test['Parch'].values.reshape(-1, 1)

# Sex
sex_train = to_categorical(le.fit_transform(df_train['Sex']))
sex_test = to_categorical(le.fit_transform(df_test['Sex']))

# Pclass
pclass_train = to_categorical(le.fit_transform(df_train['Pclass']))
pclass_test = to_categorical(le.fit_transform(df_test['Pclass']))

# Age
age_train = df_train['Age'].values.reshape(-1, 1)
age_test  = df_test['Age'].values.reshape(-1, 1)

# X and Y
num = len(df_train)
split = int(num * .75)

x_train = np.hstack([pclass_train, sex_train, age_train, siblings_train, parents_train, embarked_train])
x_train = mms.fit_transform(x_train)

y_train = df_train['Survived'].values

# X test
x_test = np.hstack([pclass_test,  sex_test,  age_test, siblings_test, parents_test, embarked_test])
x_test = mms.fit_transform(x_test)

pd.DataFrame(x_train).head()


Using TensorFlow backend.


,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,1.0,0.0,1.0,0.271174,0.2,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.472229,0.2,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.321438,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.434531,0.2,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.434531,0.0,0.0,0.0,0.0,1.0


In [4]:
"""
Define and train model
"""
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(16, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(16, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(16, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

epochs = 400
batch_size = 16

model.fit(x_train, y_train, validation_split=.1, epochs=epochs, batch_size=batch_size, shuffle=True)

Train on 640 samples, validate on 72 samples
Epoch 1/400
640/640 [==============================] - 0s - loss: 0.6763 - acc: 0.6719 - val_loss: 0.6690 - val_acc: 0.6389
Epoch 2/400
640/640 [==============================] - 0s - loss: 0.6696 - acc: 0.6625 - val_loss: 0.6613 - val_acc: 0.6389
Epoch 3/400
640/640 [==============================] - 0s - loss: 0.6632 - acc: 0.6547 - val_loss: 0.6536 - val_acc: 0.6389
Epoch 4/400
640/640 [==============================] - 0s - loss: 0.6564 - acc: 0.6687 - val_loss: 0.6452 - val_acc: 0.6389
Epoch 5/400
640/640 [==============================] - 0s - loss: 0.6494 - acc: 0.6750 - val_loss: 0.6363 - val_acc: 0.6389
Epoch 6/400
640/640 [==============================] - 0s - loss: 0.6420 - acc: 0.6797 - val_loss: 0.6273 - val_acc: 0.6806
Epoch 7/400
640/640 [==============================] - 0s - loss: 0.6342 - acc: 0.7063 - val_loss: 0.6175 - val_acc: 0.7222
Epoch 8/400
640/640 [==============================] - 0s - loss: 0.6256 - acc: 0.7281 

640/640 [==============================] - 0s - loss: 0.4248 - acc: 0.8094 - val_loss: 0.4032 - val_acc: 0.8194
Epoch 132/400
640/640 [==============================] - 0s - loss: 0.4248 - acc: 0.8078 - val_loss: 0.4067 - val_acc: 0.7917
Epoch 133/400
640/640 [==============================] - 0s - loss: 0.4243 - acc: 0.8094 - val_loss: 0.4058 - val_acc: 0.7917
Epoch 134/400
640/640 [==============================] - 0s - loss: 0.4241 - acc: 0.8109 - val_loss: 0.4001 - val_acc: 0.8333
Epoch 135/400
640/640 [==============================] - 0s - loss: 0.4253 - acc: 0.7953 - val_loss: 0.4011 - val_acc: 0.8333
Epoch 136/400
640/640 [==============================] - 0s - loss: 0.4244 - acc: 0.8109 - val_loss: 0.4055 - val_acc: 0.7917
Epoch 137/400
640/640 [==============================] - 0s - loss: 0.4241 - acc: 0.8094 - val_loss: 0.4016 - val_acc: 0.8194
Epoch 138/400
640/640 [==============================] - 0s - loss: 0.4249 - acc: 0.8031 - val_loss: 0.4018 - val_acc: 0.8194
Epoch 

640/640 [==============================] - 0s - loss: 0.4148 - acc: 0.8203 - val_loss: 0.3934 - val_acc: 0.7917
Epoch 262/400
640/640 [==============================] - 0s - loss: 0.4146 - acc: 0.8250 - val_loss: 0.3963 - val_acc: 0.7917
Epoch 263/400
640/640 [==============================] - 0s - loss: 0.4147 - acc: 0.8188 - val_loss: 0.3984 - val_acc: 0.8056
Epoch 264/400
640/640 [==============================] - 0s - loss: 0.4147 - acc: 0.8203 - val_loss: 0.3977 - val_acc: 0.7778
Epoch 265/400
640/640 [==============================] - 0s - loss: 0.4144 - acc: 0.8219 - val_loss: 0.3939 - val_acc: 0.7778
Epoch 266/400
640/640 [==============================] - 0s - loss: 0.4161 - acc: 0.8203 - val_loss: 0.3928 - val_acc: 0.7917
Epoch 267/400
640/640 [==============================] - 0s - loss: 0.4139 - acc: 0.8219 - val_loss: 0.3925 - val_acc: 0.8056
Epoch 268/400
640/640 [==============================] - 0s - loss: 0.4136 - acc: 0.8266 - val_loss: 0.4001 - val_acc: 0.8056
Epoch 

640/640 [==============================] - 0s - loss: 0.4059 - acc: 0.8328 - val_loss: 0.3949 - val_acc: 0.8194
Epoch 392/400
640/640 [==============================] - 0s - loss: 0.4059 - acc: 0.8250 - val_loss: 0.3901 - val_acc: 0.8056
Epoch 393/400
640/640 [==============================] - 0s - loss: 0.4060 - acc: 0.8281 - val_loss: 0.3880 - val_acc: 0.7917
Epoch 394/400
640/640 [==============================] - 0s - loss: 0.4038 - acc: 0.8344 - val_loss: 0.3987 - val_acc: 0.7778
Epoch 395/400
640/640 [==============================] - 0s - loss: 0.4074 - acc: 0.8188 - val_loss: 0.3980 - val_acc: 0.7917
Epoch 396/400
640/640 [==============================] - 0s - loss: 0.4055 - acc: 0.8312 - val_loss: 0.3929 - val_acc: 0.7917
Epoch 397/400
640/640 [==============================] - 0s - loss: 0.4038 - acc: 0.8297 - val_loss: 0.3883 - val_acc: 0.8056
Epoch 398/400
640/640 [==============================] - 0s - loss: 0.4065 - acc: 0.8312 - val_loss: 0.3916 - val_acc: 0.8056
Epoch 

In [69]:
"""
Create submission from model
"""
survived = (model.predict(x_test) > 0.5).astype('int')
passenger_ids = df_test['PassengerId'].values
passenger_ids = passenger_ids.reshape(len(passenger_ids), 1)

print(survived.shape)
print(passenger_ids.shape)

submission = pd.DataFrame(np.hstack([passenger_ids, survived]), columns=['PassengerId', 'Survived'])
submission.to_csv('submissions/titanic.csv', index=False)
submission.head()

(418, 1)
(418, 1)
